## Airbnb

In [ ]:
import os
import pandas as pd

# Specify the directory
directory_path = 'term_project_airbnb/'

# Get all CSV files in the directory
csv_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]

# Initialize an empty list to store DataFrames
dfs = []

# Iterate through each CSV file
for file in csv_files:
    # Build the full path of the file
    file_path = os.path.join(directory_path, file)
    
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Extract City, State, and Country from the file name
    city, state, country = file.replace('.csv', '').split(',')[:3]

    # Add attributes to the DataFrame
    df['city'] = city.strip()
    df['state'] = state.strip()
    df['country'] = country.strip()

    # Append the current DataFrame to the list
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

# Reorder columns, placing City, State, Country at the end
selected_columns = ['id', 'room_type', 'price',
                    'host_id', 'host_name', 'host_total_listings_count',
                    'calendar_last_scraped', 'review_scores_rating',
                    'city', 'state', 'country']

combined_df = combined_df[selected_columns]

# Display the result
print(combined_df)

In [ ]:
# convert the 'price' column to a numeric data type
combined_df['price'] = combined_df['price'].replace('[\$,]', '', regex=True).astype(float)
print(combined_df)

In [ ]:
import pandas as pd

# sort date for scd mainenance
combined_df = combined_df.sort_values(by=['id', 'calendar_last_scraped'])
print(combined_df.head(6))

In [ ]:
import pandas as pd

# Convert the "calendar_last_scraped" column to datetime type
combined_df['calendar_last_scraped'] = pd.to_datetime(combined_df['calendar_last_scraped'])

# Extract year, month, and day information
combined_df['year'] = combined_df['calendar_last_scraped'].dt.year
combined_df['quarter'] = combined_df['calendar_last_scraped'].dt.quarter
combined_df['month'] = combined_df['calendar_last_scraped'].dt.month
combined_df['day'] = combined_df['calendar_last_scraped'].dt.day

# Print the modified DataFrame
print(combined_df.head(6))


In [ ]:
print("Unique calendar_last_scraped:")
print(combined_df[['year', 'month', 'day']].drop_duplicates())

print("\nUnique City State Country:")
print(combined_df[['city', 'state', 'country']].drop_duplicates())

## LinkedIn

In [ ]:
import pandas as pd

job_postings = pd.read_csv('job_postings.csv')
company_details = pd.read_csv('companies.csv')

merged_data = pd.merge(job_postings, company_details, on='company_id', how='left')
merged_data.to_csv('job_companies.csv', index=False)

In [ ]:
import pandas as pd

columns_to_read = [
    'job_id',
    'max_salary',
    'med_salary',
    'min_salary',
    'expiry',
    'formatted_experience_level',
    'listed_time',
    'sponsored',
    'work_type',
    'name',
    'company_size',
    'state',
    'country',
    'city'
]

jobs_df = pd.read_csv('job_companies.csv', usecols=columns_to_read)

In [ ]:
print("Number of rows in jobs_df:", len(jobs_df))
jobs_df = jobs_df[(jobs_df['country'].notna()) & (jobs_df['country'] != '0') & (jobs_df['state'] != '0')]
print("Number of rows in jobs_df:", len(jobs_df))

In [ ]:
print("Unique countries:", jobs_df['country'].unique())

In [ ]:
import re
import pandas as pd

# Mapping for state abbreviations
state_mapping = {
    'CALIFORNIA': 'CA',
    'NEWYORK': 'NY',
    'TEXAS': 'TX',
    'MASSACHUSETTS': 'MA',
    'MONTANA': 'MT',
    'FLORIDA': 'FL',
    'ILLINOIS': 'IL',
    'NEVADA': 'NV',
    'OHIO': 'OH',
    'COLORADO': 'CO',
    'HAWAII': 'HI',
    'NEWJERSEY': 'NJ',
    'TENNESSEE': 'TN',
    'LOUISIANA': 'LA',
    'OREGON': 'OR',
    'RHODEISLAND': 'RI',
    'WASHINGTON': 'WA',
    'MINNESOTA': 'MN',
    'DISTRICTOFCOLUMBIA': 'DC',
}

# Assume you have already created the jobs_df DataFrame
# If not, you can use the previously mentioned code to create it

# Create a new DataFrame to store the processed state values
processed_jobs_df = jobs_df.copy()

# Preprocess state names: remove spaces, special characters, and convert to uppercase
processed_jobs_df['state'] = processed_jobs_df['state'].apply(lambda x: re.sub(r'\W+', '', str(x).upper()))

# Apply the state mapping with a default value for unknown states
processed_jobs_df['state'] = processed_jobs_df['state'].map(lambda x: state_mapping.get(x, x))

# Filter DataFrame to include only rows with valid state mappings
valid_states_df = processed_jobs_df[processed_jobs_df['state'].isin(state_mapping.values())]

# Count the occurrences of each state and print in the specified order
state_counts = valid_states_df[valid_states_df['country'] == 'US']['state'].value_counts()

# Print counts in the specified order
for state, count in state_counts.items():
    print(f"{state}: {count}")

In [ ]:
valid_states_df['salary'] = valid_states_df.apply(lambda row: row['med_salary'] if not pd.isnull(row['med_salary'])
                                                  else (None if pd.isnull(row['min_salary']) and pd.isnull(row['max_salary'])
                                                        else (row['min_salary'] + row['max_salary']) / 2), axis=1)

In [ ]:
# Convert 'listed_time' and 'expiry' to datetime and extract components
valid_states_df['listed_time'] = pd.to_datetime(valid_states_df['listed_time'], unit='ms')
valid_states_df['expiry'] = pd.to_datetime(valid_states_df['expiry'], unit='ms')

# Extract components for 'listed_time'
valid_states_df['list_year'] = valid_states_df['listed_time'].dt.year
valid_states_df['list_month'] = valid_states_df['listed_time'].dt.month
valid_states_df['list_day'] = valid_states_df['listed_time'].dt.day

# Extract components for 'expiry'
valid_states_df['expiry_year'] = valid_states_df['expiry'].dt.year
valid_states_df['expiry_month'] = valid_states_df['expiry'].dt.month
valid_states_df['expiry_day'] = valid_states_df['expiry'].dt.day

print(valid_states_df)

## Load

In [ ]:
combined_df_copy = combined_df.copy()
valid_states_df_copy = valid_states_df.copy()

In [ ]:
combined_df = combined_df_copy.copy()
valid_states_df = valid_states_df_copy.copy()

In [ ]:
print(combined_df)
print(valid_states_df)

In [ ]:
combined_df.dropna(subset=['host_total_listings_count'], inplace=True)
combined_df['host_total_listings_count'] = combined_df['host_total_listings_count'].astype(int)

### location

In [ ]:
# Extract City, State, Country from combined_df
combined_locations = combined_df[['city', 'state', 'country']].copy()
combined_locations = combined_locations.drop_duplicates()
#print(combined_locations)
# Extract state, country, city from valid_states_df
valid_states_locations = valid_states_df[['state', 'country', 'city']].copy()
valid_states_locations = valid_states_locations.drop_duplicates()
#print(valid_states_locations)

# Combine all locations and create a unique ID starting from 0
all_locations = pd.concat([combined_locations, valid_states_locations], ignore_index=True)
all_locations = all_locations.drop_duplicates()

# Add a unique ID column starting from 0
#all_locations['location_id'] = range(len(all_locations))
all_locations['location_id'] = range(1, len(all_locations) + 1)

# Print the resulting DataFrame with unique ID starting from 0
print(all_locations)

In [ ]:
# Merge location_id with combined_df
combined_df = pd.merge(combined_df, all_locations, how='left', on=['city', 'state', 'country'])

# Merge location_id with valid_states_df
valid_states_df = pd.merge(valid_states_df, all_locations, how='left', left_on=['city', 'state', 'country'], right_on=['city', 'state', 'country'])

In [ ]:
# Print 'city', 'state', 'country' columns in combined_df
print(combined_df[['city', 'state', 'country', 'location_id']])

# Print 'city', 'state', 'country' columns in valid_states_df
print(valid_states_df[['city', 'state', 'country', 'location_id']])

#### load

In [ ]:
import pyodbc
import pandas as pd

conn_str = 'DRIVER={SQL Server};SERVER=DESKTOP-14JKRFU\\MSSQLSERVER01;DATABASE=term;Trusted_Connection=yes;'

conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

for index, row in all_locations.iterrows():
    cursor.execute("INSERT INTO Location (id, country, state, city) VALUES (?, ?, ?, ?)",
                   row['location_id'], row['country'], row['state'], row['city'])


conn.commit()

conn.close()

### time

In [ ]:
print(combined_df[['year', 'quarter', 'month', 'day']])
print(valid_states_df[['list_year', 'list_month', 'list_day', 'expiry_year', 'expiry_month', 'expiry_day']])

In [ ]:
# Extract 'year', 'month', and 'day' columns from both DataFrames
combined_dates = combined_df[['year', 'month', 'day']]
valid_states_list = valid_states_df[['list_year', 'list_month', 'list_day']].rename(columns={'list_year': 'year', 'list_month': 'month', 'list_day': 'day'})
valid_states_expiry = valid_states_df[['expiry_year', 'expiry_month', 'expiry_day']].rename(columns={'expiry_year': 'year', 'expiry_month': 'month', 'expiry_day': 'day'})

# Concatenate the DataFrames vertically
all_dates = pd.concat([combined_dates, valid_states_list, valid_states_expiry])

# Remove duplicate rows based on 'year', 'month', and 'day'
all_dates_no_duplicates = all_dates.drop_duplicates(subset=['year', 'month', 'day'])

# Sort the DataFrame by 'year', 'month', and 'day'
all_dates_sorted = all_dates_no_duplicates.sort_values(by=['year', 'month', 'day'])

# Add a new column 'date_id' with unique values starting from 1
all_dates_sorted['date_id'] = range(1, len(all_dates_sorted) + 1)

print(all_dates_sorted)


In [ ]:
import pyodbc
import pandas as pd

conn_str = 'DRIVER={SQL Server};SERVER=DESKTOP-14JKRFU\\MSSQLSERVER01;DATABASE=term;Trusted_Connection=yes;'

conn = pyodbc.connect(conn_str)
cursor = conn.cursor()


for index, row in all_dates_sorted.iterrows():
    cursor.execute("INSERT INTO DayTime (id, year, month, day) VALUES (?, ?, ?, ?)",
               int(row['date_id']), int(row['year']), int(row['month']), int(row['day']))



conn.commit()
conn.close()

In [ ]:
# Merge date_id with combined_df
combined_df = pd.merge(combined_df, all_dates_sorted, how='left', on=['year', 'month', 'day'])

# Merge date_id with valid_states_df for list_date_id
valid_states_df = pd.merge(valid_states_df, all_dates_sorted, how='left', left_on=['list_year', 'list_month', 'list_day'], right_on=['year', 'month', 'day'])
valid_states_df = valid_states_df.rename(columns={'date_id': 'list_date_id'})

# Merge date_id with valid_states_df for expiry_date_id
valid_states_df = pd.merge(valid_states_df, all_dates_sorted, how='left', left_on=['expiry_year', 'expiry_month', 'expiry_day'], right_on=['year', 'month', 'day'])
valid_states_df = valid_states_df.rename(columns={'date_id': 'expiry_date_id'})


In [ ]:
print(combined_df[['year', 'month', 'day', 'date_id']])
print(valid_states_df[['list_year', 'list_month', 'list_day', 'list_date_id']])
print(valid_states_df[['expiry_year', 'expiry_month', 'expiry_day', 'expiry_date_id']])

In [ ]:
print(combined_df[['year', 'quarter', 'month']])

In [ ]:
# Extract 'list_year', 'list_quarter', 'list_month' columns from valid_states_df
list_months = combined_df[['year', 'quarter', 'month']]

# Remove duplicate rows based on 'list_year', 'list_quarter', and 'list_month'
unique_list_months = list_months.drop_duplicates()

# Sort the DataFrame by 'list_year', 'list_quarter', and 'list_month'
sorted_list_months = unique_list_months.sort_values(by=['year', 'quarter', 'month'])

# Add a new column 'list_month_id' with unique values starting from 1
sorted_list_months['month_id'] = range(1, len(sorted_list_months) + 1)

# Merge 'list_month_id' with valid_states_df
combined_df = pd.merge(combined_df, sorted_list_months, how='left', on=['year', 'quarter', 'month'])

In [ ]:
print(sorted_list_months)
#print(valid_states_df['list_month_id'])

In [ ]:
import pyodbc
import pandas as pd

conn_str = 'DRIVER={SQL Server};SERVER=DESKTOP-14JKRFU\\MSSQLSERVER01;DATABASE=term;Trusted_Connection=yes;'

conn = pyodbc.connect(conn_str)
cursor = conn.cursor()


for index, row in sorted_list_months.iterrows():
    cursor.execute("INSERT INTO MonthTime (id, year, quarter, month) VALUES (?, ?, ?, ?)",
               int(row['month_id']), int(row['year']), int(row['quarter']), int(row['month']))



conn.commit()
conn.close()

### company SCD 1

In [ ]:
companies = valid_states_df[['name', 'company_size']]
unique_companies = companies.drop_duplicates()
sorted_companies = unique_companies.sort_values(by=['name', 'company_size'])

if 'company_id' in valid_states_df.columns:
    valid_states_df = valid_states_df.drop(columns=['company_id'])

sorted_companies['company_id'] = range(1, len(sorted_companies) + 1)

In [ ]:
print(sorted_companies)
print(valid_states_df)

In [ ]:
valid_states_df = pd.merge(valid_states_df, sorted_companies, how='left', on=['name', 'company_size'])

In [ ]:
import pyodbc
import pandas as pd
import numpy as np

conn_str = 'DRIVER={SQL Server};SERVER=DESKTOP-14JKRFU\\MSSQLSERVER01;DATABASE=term;Trusted_Connection=yes;'

conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

for index, row in sorted_companies.iterrows():

    company_id = int(row['company_id']) if not pd.isna(row['company_id']) else None
    company_size = int(row['company_size']) if not pd.isna(row['company_size']) else None
    
    cursor.execute("INSERT INTO Company (id, name, company_size) VALUES (?, ?, ?)",
                   company_id, row['name'], company_size)

conn.commit()
conn.close()


### job

In [ ]:
# Extract 'work_type', 'formatted_experience_level', 'sponsored' columns from valid_states_df
job_description_columns = valid_states_df[['work_type', 'formatted_experience_level', 'sponsored']]

# Remove duplicate rows based on the selected columns
unique_job_descriptions = job_description_columns.drop_duplicates()

# Add a new column 'job_description_id' with unique values starting from 1
unique_job_descriptions['job_description_id'] = range(1, len(unique_job_descriptions) + 1)

# Merge 'job_description_id' with valid_states_df
valid_states_df = pd.merge(valid_states_df, unique_job_descriptions, how='left', on=['work_type', 'formatted_experience_level', 'sponsored'])
print(unique_job_descriptions)

In [ ]:
import pyodbc
import pandas as pd

conn_str = 'DRIVER={SQL Server};SERVER=DESKTOP-14JKRFU\\MSSQLSERVER01;DATABASE=term;Trusted_Connection=yes;'

conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

for index, row in unique_job_descriptions.iterrows():
    
    formatted_experience_level = str(row['formatted_experience_level']) if pd.notna(row['formatted_experience_level']) else ""
    
    cursor.execute("INSERT INTO Job_description (id, work_type, experience_level_required, sponsored) VALUES (?, ?, ?, ?)",
                   int(row['job_description_id']), row['work_type'], formatted_experience_level, int(row['sponsored']))

conn.commit()
conn.close()


### room SCD 2

#### for test purpose only!!!!

In [1]:
import pyodbc
import pandas as pd

# Establish connection
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=DESKTOP-14JKRFU\\MSSQLSERVER01;DATABASE=term_test;Trusted_Connection=yes;')
cursor = conn.cursor()

# SQL query to fetch data
sql_query = """
SELECT
    [room_id],
    [airbnb_key],
    [room_type],
    [price],
    [effect_date],
    [expire_date],
    [current_flag]
FROM
    [term_test].[dbo].[Room]
"""

# Execute query and load data into Pandas DataFrame
staging_room_df = pd.read_sql_query(sql_query, conn)

# Close the database connection
conn.close()

# Display the DataFrame
print(staging_room_df)


   room_id           airbnb_key     room_type  price effect_date expire_date  \
0        1  1016619106223207155  Entire place  349.0  2023-09-18  9999-12-31   
1        1  1016619106223207155  Entire place  300.0  2023-08-18  2023-09-17   

  current_flag  
0            Y  
1            N  


C:\Users\ALIENWARE\AppData\Local\Temp\ipykernel_33612\4012980232.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  staging_room_df = pd.read_sql_query(sql_query, conn)


In [2]:
import pandas as pd

# Test delta data
test_df_data = {
    'id': ['1', '2', '1', '2', '1', '1016619106223207155'],
    'calendar_last_scraped': ['2023-01-01', '2023-01-02', '2023-01-05', '2023-01-07', '2023-01-10', '2023-01-10'],
    'room_type': ['Entire place', 'Private room', 'Entire place', 'Private room', 'Shared room', 'Shared room'],
    'price': [100.00, 75.00, 110.00, 75.00, 50.00, 350.00]
}

# Create a DataFrame
test_df = pd.DataFrame(test_df_data)

# Display the DataFrame
print(test_df)

# Display the data types of each column
print(test_df.dtypes)

                    id calendar_last_scraped     room_type  price
0                    1            2023-01-01  Entire place  100.0
1                    2            2023-01-02  Private room   75.0
2                    1            2023-01-05  Entire place  110.0
3                    2            2023-01-07  Private room   75.0
4                    1            2023-01-10   Shared room   50.0
5  1016619106223207155            2023-01-10   Shared room  350.0
id                        object
calendar_last_scraped     object
room_type                 object
price                    float64
dtype: object


In [3]:
# Initialize an empty dictionary as a map
room_map = {}

# Iterate through each row in staging_room_df and map airbnb_key to room_id
for index, row in staging_room_df.iterrows():
    airbnb_key = row['airbnb_key']
    room_id = row['room_id']
    room_map[airbnb_key] = room_id

# Print the contents of the dictionary to check if it's correct
print(room_map)

{'1016619106223207155': 1}


In [4]:
#room_map = {}

# Use the default parameter to handle an empty dictionary
max_value = max(set(room_map.values()), default=None)

if max_value is None:
    max_value = 0

count = max_value + 1

distinct_ids = set(test_df['id'].unique())

# Subtract the set of room_map keys from distinct_ids
result_set = distinct_ids - set(room_map.keys())
result_set = sorted(result_set)
print(result_set)

for key in result_set:
    room_map[key] = count
    count += 1

print(room_map)

['1', '2']
{'1016619106223207155': 1, '1': 2, '2': 3}


In [5]:
test_df['room_id'] = test_df['id'].map(room_map)
print(test_df)

                    id calendar_last_scraped     room_type  price  room_id
0                    1            2023-01-01  Entire place  100.0        2
1                    2            2023-01-02  Private room   75.0        3
2                    1            2023-01-05  Entire place  110.0        2
3                    2            2023-01-07  Private room   75.0        3
4                    1            2023-01-10   Shared room   50.0        2
5  1016619106223207155            2023-01-10   Shared room  350.0        1


In [6]:
from datetime import timedelta
import pandas as pd

# Assuming test_df is your DataFrame
# You need to replace 'id' with the actual column name in your DataFrame
# test_df['room_id'] = test_df['id'].map(room_map)

process_df = test_df
# process_df = pd.DataFrame(combined_df)

# convert the 'calendar_last_scraped' column to date format and 
# sort the DataFrame based on 'room_id' and in chronological order for (SCD2) maintenance.
process_df['calendar_last_scraped'] = pd.to_datetime(process_df['calendar_last_scraped'])
process_df_sorted = process_df.sort_values(by=['room_id', 'calendar_last_scraped'])

# initialize a dictionary (room_records) to store room records and an index (room_index) for the recent updated index.
room_records = {}
room_index = {}

# Populate room_records with initial records from staging_room_df
# Assuming staging_room_df is your DataFrame containing room_id, airbnb_key, room_type, price, etc.
for index, row in staging_room_df.iterrows():
    room_records[len(room_records) + 1] = {
        'room_id': row['room_id'],
        'airbnb_key': row['airbnb_key'],
        'room_type': row['room_type'],
        'price': row['price'],
        'effect_date': row['effect_date'],
        'expire_date': row['expire_date'],
        'current_flag': row['current_flag']
    }
    room_index[(row['room_id'], row['current_flag'])] = room_records[len(room_records)]



In [7]:
# Iterating through the sorted DataFrame
for index, row in process_df_sorted.iterrows():
    unique_id = row['room_id']
    airbnb_key = row['id']
    current_flag = 'Y'

    # Check if there is an existing record with the same room_id and current_flag 'Y' using the index
    existing_record = room_index.get((unique_id, current_flag))

    if existing_record:
        # Existing record found, check if relevant columns are the same
        if (
            row['room_type'] == existing_record['room_type'] and
            row['price'] == existing_record['price']
        ):
            continue
        else:
            # Relevant columns are different, expire the existing record and insert a new one
            expire_date = row['calendar_last_scraped'].strftime("%Y-%m-%d")
            existing_record['expire_date'] = expire_date
            existing_record['current_flag'] = 'N'

    # No existing record found or existing record has current_flag 'N', it's a new record
    effect_date = row['calendar_last_scraped'].strftime("%Y-%m-%d")
    expire_date = '9999-12-31'

    # Create a new record in room_records and update the index
    room_records[len(room_records) + 1] = {
        'room_id': unique_id,
        'airbnb_key': airbnb_key,
        'room_type': row['room_type'],
        'price': row['price'],
        'effect_date': effect_date,
        'expire_date': expire_date,
        'current_flag': current_flag
    }
    room_index[(unique_id, current_flag)] = room_records[len(room_records)]

# Create a DataFrame from room_records
room_df = pd.DataFrame(list(room_records.values()))

# Print the resulting DataFrame
print(room_df)


   room_id           airbnb_key     room_type  price effect_date expire_date  \
0        1  1016619106223207155  Entire place  349.0  2023-09-18  2023-01-10   
1        1  1016619106223207155  Entire place  300.0  2023-08-18  2023-09-17   
2        1  1016619106223207155   Shared room  350.0  2023-01-10  9999-12-31   
3        2                    1  Entire place  100.0  2023-01-01  2023-01-05   
4        2                    1  Entire place  110.0  2023-01-05  2023-01-10   
5        2                    1   Shared room   50.0  2023-01-10  9999-12-31   
6        3                    2  Private room   75.0  2023-01-02  9999-12-31   

  current_flag  
0            N  
1            N  
2            Y  
3            N  
4            N  
5            Y  
6            Y  


In [9]:
import pyodbc

# Create a connection and cursor
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=DESKTOP-14JKRFU\\MSSQLSERVER01;DATABASE=term_test;Trusted_Connection=yes;')
cursor = conn.cursor()

# Delete all records from the Room table
cursor.execute("DELETE FROM Room")

# Commit changes
conn.commit()

# Insert data into the Room table
for index, row in room_df.iterrows():
    cursor.execute("""
        INSERT INTO Room (room_id, airbnb_key, room_type, price, effect_date, expire_date, current_flag)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """, row['room_id'], row['airbnb_key'], row['room_type'], row['price'], row['effect_date'], row['expire_date'], row['current_flag'])

# Commit changes
conn.commit()

# Close the connection
conn.close()


#### prev

In [ ]:
import pandas as pd

new_id_df = combined_df[['id']].copy()
new_id_df = new_id_df.drop_duplicates().reset_index(drop=True)
new_id_df['new_id'] = range(1, len(new_id_df) + 1)
print(new_id_df)

In [ ]:
combined_df = pd.merge(combined_df, new_id_df, how='left', on=['id'])
combined_df = combined_df.drop(columns=['id'])
combined_df = combined_df.rename(columns={'new_id': 'id'})

In [ ]:
print(len(combined_df))
print(len(combined_df['id'].unique()))

In [ ]:
combined_df.rename(columns={'id': 'room_id'}, inplace=True)

#### test

In [ ]:
import pandas as pd
# test delta data
test_data = {'room_id': [1, 2, 1, 2, 1, 3],
        'calendar_last_scraped': ['2023-01-01', '2023-01-02', '2023-01-05', '2023-01-07', '2023-01-10', '2023-01-10'],
        'room_type': ['Entire place', 'Private room', 'Entire place', 'Private room', 'Shared room', 'Shared room'],
        'price': [100.00, 75.00, 110.00, 75.00, 50.00, 350.00]}

test_df = pd.DataFrame(test_data)
print(test_df)

In [ ]:
from datetime import timedelta
import pandas as pd

process_df = pd.DataFrame(test_df)
#process_df = pd.DataFrame(combined_df)

# convert the 'calendar_last_scraped' column to date format and 
#sort the DataFrame based on 'room_id' and in chronological order for (SCD2) maintenance.
process_df['calendar_last_scraped'] = pd.to_datetime(process_df['calendar_last_scraped'])
process_df_sorted = process_df.sort_values(by=['room_id', 'calendar_last_scraped'])

# initialize a dictionary (room_records) to store room records and an index (room_index) for the recent updated index.
room_records = {}
room_index = {}

# Iterating through the sorted DataFramecheck 
for index, row in process_df_sorted.iterrows():
    unique_id = row['room_id']
    current_flag = 'Y'

    # Check if there is an existing record with the same room_id and current_flag 'Y' using the index
    existing_record = room_index.get((unique_id, current_flag))

    if existing_record:
        # Existing record found, check if relevant columns are the same
        if (
            row['room_type'] == existing_record['room_type'] and
            row['price'] == existing_record['price']
        ):
            continue
        else:
            # Relevant columns are different, expire the existing record and insert a new one
            expire_date = row['calendar_last_scraped'].strftime("%Y-%m-%d")
            existing_record['expire_date'] = expire_date
            existing_record['current_flag'] = 'N'

    # No existing record found or existing record has current_flag 'N', it's a new record
    effect_date = row['calendar_last_scraped']
    expire_date = '9999-12-31'

    # Create a new record in room_records and update the index
    room_records[len(room_records) + 1] = {
        'room_id': unique_id,
        'room_type': row['room_type'],
        'price': row['price'],
        'effect_date': effect_date,
        'expire_date': expire_date,
        'current_flag': current_flag
    }
    room_index[(unique_id, current_flag)] = room_records[len(room_records)]

# Create a DataFrame from room_records
room_df = pd.DataFrame(list(room_records.values()))

# Print the resulting DataFrame
print(room_df)


In [ ]:
print(len(room_df))
print(len(room_df['room_id'].unique()))

In [ ]:
import pyodbc

# Create a connection and cursor
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=DESKTOP-14JKRFU\\MSSQLSERVER01;DATABASE=term;Trusted_Connection=yes;')
cursor = conn.cursor()

# Insert data into the Room table
for index, row in room_df.iterrows():
    cursor.execute("""
        INSERT INTO Room (room_id, room_type, price, effect_date, expire_date, current_flag)
        VALUES (?, ?, ?, ?, ?, ?)
    """, row['room_id'], row['room_type'], row['price'], row['effect_date'], row['expire_date'], row['current_flag'])

# Commit changes
conn.commit()

# Close the connection
conn.close()

### host SCD 3

In [ ]:
# Create a new DataFrame for host_id
new_host_id_df = combined_df[['host_id']].copy()
new_host_id_df = new_host_id_df.drop_duplicates().reset_index(drop=True)
new_host_id_df['new_host_id'] = range(1, len(new_host_id_df) + 1)
print(new_host_id_df)

# Merge the new_host_id_df with combined_df
combined_df = pd.merge(combined_df, new_host_id_df, how='left', on=['host_id'])

# Drop the original host_id column
combined_df = combined_df.drop(columns=['host_id'])

# Rename the new_host_id column to host_id
combined_df = combined_df.rename(columns={'new_host_id': 'host_id'})

# Print the updated DataFrame
print(combined_df)


#### test

In [ ]:
# test delta data
test_host_data = {'host_id': [1, 2, 1, 2, 1, 3],
        'calendar_last_scraped': ['2023-01-01', '2023-01-02', '2023-01-05', '2023-01-07', '2023-01-10', '2023-01-10'],
        'host_name': ['A', 'B', 'A', 'B', 'C', 'C'],
        'host_total_listings_count': [100, 75, 110, 75, 50, 350]}

test_host_df = pd.DataFrame(test_host_data)
print(test_host_df)

In [ ]:
from datetime import timedelta
import pandas as pd

process_df = pd.DataFrame(test_host_df)
#process_df = pd.DataFrame(combined_df)

# Convert 'calendar_last_scraped' to datetime
# Sort the DataFrame by 'host_id' and 'calendar_last_scraped' for SCD maintenance
process_df['calendar_last_scraped'] = pd.to_datetime(process_df['calendar_last_scraped'])
process_df_sorted = process_df.sort_values(by=['host_id', 'calendar_last_scraped'])

# initialize a (host_records) to store host records and an (host_index) for the recent updated index.
host_records = {}
host_index = {}

# Iterate through each row in process_df_sorted
for index, row in process_df_sorted.iterrows():
    unique_id = row['host_id']

    # Check if there is an existing record with the same host_id using the index
    existing_record = host_index.get(unique_id)

    if existing_record:
        # Existing record found, check if relevant columns are the same
        if row['host_name'] != existing_record['host_name']:
            # Relevant column 'host_name' has changed, update the existing record with historical values
            timestamp = row["calendar_last_scraped"].strftime('%Y-%m')
            existing_record[f'host_name_{timestamp}'] = existing_record['host_name']
            existing_record['host_name'] = row['host_name']

        if row['host_total_listings_count'] != existing_record['host_total_listings_count']:
            # Relevant column 'host_total_listings_count' has changed, update the existing record with historical values
            timestamp = row["calendar_last_scraped"].strftime('%Y-%m')
            existing_record[f'host_total_listings_count_{timestamp}'] = existing_record['host_total_listings_count']
            existing_record['host_total_listings_count'] = row['host_total_listings_count']

    else:
        # No existing record found, it's a new record
        host_records[unique_id] = {
            'host_id': unique_id,
            'host_name': row['host_name'],
            'host_total_listings_count': row['host_total_listings_count']
        }
        host_index[unique_id] = host_records[len(host_records)]

# Create a DataFrame from host_records
host_df = pd.DataFrame(list(host_records.values()))
host_df['host_total_listings_count'] = host_df['host_total_listings_count'].astype('int64')

# Print the resulting DataFrame
print(host_df)


In [ ]:
print(host_df.columns)
print(host_df.dtypes)

In [ ]:
for column in host_df.columns:
    if host_df[column].dtype == 'O':  # 'O' represents object (string) type
        host_df[column] = host_df[column].str.replace(',', ' ')

In [ ]:
host_df.to_csv('Host.csv', index=False)

## cumulative

In [ ]:
import pandas as pd

# Assuming combined_df is your DataFrame
# Aggregate data to calculate average_rent and total_counts
summary_df = combined_df.groupby(['month_id', 'location_id']).agg(
    average_rent=('price', 'mean'),
    total_counts=('room_id', 'count')
).reset_index()

# Create a DataFrame for monthlyRoomReservationSummary
monthly_summary_fact = pd.DataFrame({
    'month_id': summary_df['month_id'],
    'location_id': summary_df['location_id'],
    'average_rent': summary_df['average_rent'],
    'total_counts': summary_df['total_counts']
})

# Print the resulting DataFrame
print(monthly_summary_fact)


In [ ]:
import pyodbc

# Create a connection and cursor
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=DESKTOP-14JKRFU\\MSSQLSERVER01;DATABASE=term;Trusted_Connection=yes;')
cursor = conn.cursor()

# Insert data into the Room table
for index, row in monthly_summary_fact.iterrows():
    cursor.execute("""
        INSERT INTO MonthlyRoomReservationSummary (month_id, location_id, average_rent, total_counts)
        VALUES (?, ?, ?, ?)
    """, row['month_id'], row['location_id'], row['average_rent'], row['total_counts'])

# Commit changes
conn.commit()

# Close the connection
conn.close()

In [ ]:
print(combined_df)

In [ ]:
import pyodbc

# Create a connection and cursor
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=DESKTOP-14JKRFU\\MSSQLSERVER01;DATABASE=term;Trusted_Connection=yes;')
cursor = conn.cursor()

# Insert data into the Room table
for index, row in combined_df.iterrows():
    # Replace NaN with None in the 'review_scores_rating' column
    row['review_scores_rating'] = row['review_scores_rating'] if pd.notnull(row['review_scores_rating']) else None
    
    cursor.execute("""
        INSERT INTO List (location_id, time_id, room_id, host_id, review_rating)
        VALUES (?, ?, ?, ?, ?)
    """, row['location_id'], row['date_id'], row['room_id'], row['host_id'], row['review_scores_rating'])


# Commit changes
conn.commit()

# Close the connection
conn.close()

In [ ]:
print(valid_states_df.columns)

In [ ]:
print(valid_states_df[['location_id', 'job_description_id', 'company_id', 'expiry_date_id', 'list_date_id', 'salary']])

In [ ]:
import pyodbc

# Create a connection and cursor
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=DESKTOP-14JKRFU\\MSSQLSERVER01;DATABASE=term;Trusted_Connection=yes;')
cursor = conn.cursor()

# Insert data into the Room table
for index, row in valid_states_df.iterrows():
    row['salary'] = row['salary'] if pd.notnull(row['salary']) else None
    cursor.execute("""
        INSERT INTO Job (location_id, job_description_id, company_id, expiry, listed_time, med_salary)
        VALUES (?, ?, ?, ?, ?, ?)
    """, row['location_id'], row['job_description_id'], row['company_id'], row['expiry_date_id'], row['list_date_id'], row['salary'])


# Commit changes
conn.commit()

# Close the connection
conn.close()

# alternative code